In [1]:
#This is prs script where we can call the CI script in a loop getting a list of percentiles 
#This is with replacement, and with masking out alleles that are not found in the individual


In [13]:
#reading in risk allele file
riskallelefile = open("PGS000004.txt", "r")
riskallele=[]
for line in riskallelefile:
    if line.startswith("#"):
        continue
    if line.startswith("chr"):
        continue
    else:
        row = line.strip().split("\t")
        riskallele.append(row)

In [14]:
#reading in rsid file pulled from kaviar
rsidfile = open("mavaddatrsids.txt", "r")
rsid=[]
for line in rsidfile:
    if line.startswith("#"):
        continue
    if line.startswith("c"):
        continue
    else:
        row = line.strip().split("\t")
        rsid.append(row)

In [15]:
print("creating rsid list")

#joining kaviar snps files with risk allele file, replacing chrom pos with rsid
rsidlist_all=[]       
for i in riskallele: 
    for j in rsid:
        if [i[0], i[1]] ==  [j[0], j[1]]:
            if [i[3], i[2]] == [j[3], j[4]]:
                rsidlist_all.append([j[2], i[2], i[3], i[4]])
                
#removing rsid duplicates
rsidlist=[]       
rsids=[]    
for i in rsidlist_all:   
    if i[0] in rsids:
        continue
    else:
        rsidlist.append(i)
    rsids.append(i[0])


creating rsid list


In [17]:
#reading in the vcf file 
file1open = open("new225.vcf", "r")
vcffile=[]
individuals=[]
for line in file1open:
    if line.startswith("##"):
        continue
    if line.startswith("#CHR"): 
        inds = line.strip().split("\t")
        individuals.append(inds)
        vcffile.append(inds)
    row = line.strip().split("\t")
    if row[2] in rsids:
        if row[2] not in vcffile:
            vcffile.append(row)
print("number of risk alleles found in file: ", len(vcffile))

number of risk alleles found in file:  61


In [18]:
vcf=[]
unique=[]
for i in vcffile:
    if i[2] in unique:
        continue
    else:
        vcf.append(i)
        unique.append(i[2])
print("number of unique rsids: ", len(vcf))


number of unique rsids:  61


In [19]:
#find number of snps available in sample

index = int(vcf[0].index("Falkenstein"))
individual_snps = []
for i in range(1, len(vcf)):
    if vcf[i][index] == "./.":
        continue
    else:
        individual_snps.append(vcf[i][2])

ind_snps_vcf = []
for i in vcf:
    if i[2] in individual_snps:
        ind_snps_vcf.append(i)


In [20]:
#prs function
def calculate_prs(vcf_file):
    scores=[]
    null=0
    genotype=0
    samplenumber=0
    for j in range(9, len(inds)):
            sample=[]
            totalscore=0
            #iterate through risk alleles
            for i in range(1, len(vcf_file)):
                #sample contains rsid, reference allele, alternate allele, and genotype
                sample.append([vcf_file[i][2], vcf_file[i][3], vcf_file[i][4], vcf_file[i][j]])
            for i in sample:
                for x in rsidlist:
                    if i[0] == x[0]:
                        #checking if ref allele and alt allele match
                        if i[1]==x[2] and i[2]==x[1]:
                            if i[3]!="./.": 
                                if i[3] == "0/0": 
                                    genotype = 0
                                if i[3] == "0/1" or "1/0": 
                                    genotype = 1 
                                if i[3] == "1/1": 
                                    genotype = 2 
                                score=float(x[3])* float(genotype)
                                totalscore=score + totalscore
                            else:
                                null+=1
                        #if ref allele and alternate allele are reversed
                        elif i[1]==x[1] and i[2]==x[2]:
                            if i[3]!="./.": 
                                if i[3] == "0/0": 
                                    genotype = 2
                                if i[3] == "0/1" or "1/0": 
                                    genotype = 1 
                                if i[3] == "1/1": 
                                    genotype = 0 
                                score=float(x[3])* float(genotype)
                                totalscore=score + totalscore
                            else:
                                null+=1
                        else:
                            print("null")
            scores.append([individuals[0][j], totalscore])
    return scores

In [21]:
#run prs function on different bootstrapped subsamples and find percentile rank of an individual for each subsample
import pandas as pd 
from random import choices
values = []

for i in range(20): 
    vcf_1 = choices(ind_snps_vcf, k=len(ind_snps_vcf))
    scorelist = calculate_prs(vcf_1)
    df = pd.DataFrame(scorelist, columns =['Name', 'Score'])
    df['Percentile Rank'] = df.Score.rank(pct = True)
    value = list(df['Percentile Rank'].loc[df['Name'] == 'Falkenstein'])
    values.append(value[0])
    print(value[0])

0.8533333333333334
0.8044444444444444
0.9533333333333334
0.5422222222222223
0.38222222222222224
0.9022222222222223
0.8977777777777778
0.9777777777777777
0.6222222222222222
0.44
0.6044444444444445
0.9955555555555555
0.5822222222222222
0.8177777777777778
0.9466666666666667
0.7377777777777778
0.49333333333333335
0.4711111111111111
1.0
0.8666666666666667


In [22]:
#create 95% confidence interval for list of percentile ranks
import numpy as np
import scipy.stats as st
st.t.interval(alpha=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values)) 



(0.6476268066166563, 0.8414843044944552)